<a href="https://colab.research.google.com/github/tejaskandath/CS4650_projects_sp2022/blob/master/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wfdb

from pandas.core import indexers
from matplotlib.text import Annotation


from wfdb import rdrecord, rdann, processing
import wfdb
from wfdb.io.annotation import ann_label_table
import numpy as np
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 161 kB 7.4 MB/s 


In [ ]:
import os
from torch.nn import TransformerEncoder
from torch.nn import TransformerEncoderLayer
from torch.nn import Embedding
import torch
import pickle
import numpy as np

In [ ]:
wfdb.io.show_ann_labels()

    label_store symbol                                    description
0             0                              Not an actual annotation
1             1      N                                    Normal beat
2             2      L                  Left bundle branch block beat
3             3      R                 Right bundle branch block beat
4             4      a                Aberrated atrial premature beat
5             5      V              Premature ventricular contraction
6             6      F          Fusion of ventricular and normal beat
7             7      J              Nodal (junctional) premature beat
8             8      A                   Atrial premature contraction
9             9      S     Premature or ectopic supraventricular beat
10           10      E                        Ventricular escape beat
11           11      j                 Nodal (junctional) escape beat
12           12      /                                     Paced beat
13           13     

In [ ]:
a_nums = np.arange(800, 895)
bad = [813, 814, 815, 816, 817, 818, 819, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839]
a_nums = [x for x in a_nums if x not in bad]
a_nums = list(map(str, a_nums))

anns = [rdann(x, extension='atr', pn_dir='svdb') for x in a_nums]

records = [rdrecord(num, smooth_frames= True, pn_dir='svdb') for num in a_nums]

heart_rate = [processing.ann2rr(num, 'atr', "svdb", as_array=True) for num in a_nums]

In [ ]:
# acceptable_labels = set(['N', 'L', 'R', 'a', 'V', 'F', 'J', 'S', 'E', 'j', '/', 'e', 'n', 'f'])
# temp = ['N', 'L', 'R', 'a', 'V', 'F', 'J', 'S', 'E', 'j', '/', 'e', 'n', 'f']
acceptable_labels = set(['V', 'S'])
temp = ['V', 'S']
label_to_num = {}
counts = {}
for label in temp:
    label_to_num[label] = len(label_to_num)
    counts[label] = 0

In [ ]:
print(len(a_nums))
WINDOW_SIZE = 100
x_data = []
y_data = []
for i in range(len(a_nums)):
    ann = anns[i]
    indices = [i for i, x in enumerate(ann.symbol) if x in acceptable_labels]
    idx = np.array(indices)
    record = records[i]
    signals0 = np.nan_to_num(record.p_signal[:, 0])
    for ind in idx:
        label_ind = ann.sample[ind]
        temp = signals0[(label_ind-int(WINDOW_SIZE/2)):(label_ind+int(WINDOW_SIZE/2))]
        if len(temp) != WINDOW_SIZE:
            continue
        x_data.append(temp)
        y_data.append(label_to_num[ann.symbol[ind]])
        counts[ann.symbol[ind]] += 1
        # print(ann.symbol[i], x)
        # plt.plot(signals0[(x-int(WINDOW_SIZE/2)):(x+int(WINDOW_SIZE/2))])
        # plt.show()
x = torch.tensor(np.array(x_data, dtype=np.float16)).to("cuda")
x = x.float()
x -= x.min()
x /= x.max()
y = torch.tensor(np.array(y_data)).to("cuda")
print(x.shape, y.shape, x.max(), x.min())
for label in counts:
    print(label, counts[label]/len(y))

78
torch.Size([22127, 100]) torch.Size([22127]) tensor(1., device='cuda:0') tensor(0., device='cuda:0')
V 0.4492701224748045
S 0.5507298775251954


In [ ]:
shuffle_order = torch.randperm(len(x))
shuffle_order = shuffle_order.tolist()
x = x[shuffle_order]
y = y[shuffle_order]

train_ind = int(len(x) * .85)
val_ind = int(len(x) * .95)
train_x = x[:train_ind].to("cuda")
train_y = y[:train_ind].to("cuda")
val_x = x[train_ind:val_ind].to("cuda")
val_y = y[train_ind:val_ind].to("cuda")
test_x = x[val_ind:].to("cuda")
test_y = y[val_ind:].to("cuda")
print(train_x.shape, val_x.shape, test_x.shape)

torch.Size([18807, 100]) torch.Size([2213, 100]) torch.Size([1107, 100])


In [ ]:
epoch_count = 300
batch_size = 64
import time
import sys

from tqdm import tqdm
from time import sleep

PATIENCE = 3
def train():
    patience = PATIENCE
    best_val_loss = 1e10
    for epoch in range(epoch_count):
        print("epoch", epoch)
        model.train()
        batch_count = len(train_x)//batch_size
        epoch_loss = 0
        epoch_hits = 0
        epoch_data_points = 0
        batches = range(batch_count)
        total_correct = 0
        train_shuffle_order = torch.randperm(len(train_x))
        train_shuffle_order = train_shuffle_order.tolist()
        epoch_x = train_x[train_shuffle_order]
        epoch_y = train_y[train_shuffle_order]
        with tqdm(total=batch_count, file=sys.stdout) as pbar:
            for batch in batches:
                x_batch = epoch_x[batch_size*batch:batch_size*(batch+1)]
                y_batch = epoch_y[batch_size*batch:batch_size*(batch+1)]
                epoch_data_points += len(x_batch)
                optimizer.zero_grad()
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                epoch_loss += loss.item()
                loss.backward()
                loss.detach()
                y_out = torch.argmax(outputs.detach(), axis=1)
                correct = (y_out == y_batch).sum()
                total_correct += correct
                optimizer.step()
                to_print = "epoch_loss: {loss}, epoch_accuracy: {acc}".format(
                    loss="%.2f" % (epoch_loss/(batch + 1)), acc="%.2f" % (100 * total_correct / (batch_size * (batch + 1))))
                pbar.set_description(to_print)
                pbar.update(1)
        print("final epoch_loss", "%.2f" % (epoch_loss/(batch + 1)), "final epoch_acc", "%.2f" % (100 * total_correct / len(train_x)))
        val_loss = val()
        if val_loss > best_val_loss:
            patience -= 1
        else:
            best_val_loss = val_loss
            patience = PATIENCE
        print(best_val_loss, "patience", patience)
        if patience < 0:
            break
        
def test():
    model.eval()
    batch_count = len(test_x)//batch_size
    epoch_loss = 0
    epoch_hits = 0
    epoch_data_points = 0
    total_correct = 0
    for batch in range(batch_count):
        x_batch = test_x[batch_size*batch:batch_size*(batch+1)]
        y_batch = test_y[batch_size*batch:batch_size*(batch+1)]
        epoch_data_points += len(x_batch)
        optimizer.zero_grad()
        outputs = model(x_batch)
        y_out = torch.argmax(outputs.detach(), axis=1)
        correct = (y_out == y_batch).sum()
        total_correct += correct
        loss = criterion(outputs, y_batch)
        epoch_loss += loss.item()
    print("test_loss", "%.2f" % (epoch_loss/(batch + 1)), "test_acc", "%.2f" % (100 * total_correct / len(test_x)))

def val():
    model.eval()
    batch_count = len(val_x)//batch_size
    epoch_loss = 0
    epoch_hits = 0
    epoch_data_points = 0
    total_correct = 0
    for batch in range(batch_count):
        x_batch = val_x[batch_size*batch:batch_size*(batch+1)]
        y_batch = val_y[batch_size*batch:batch_size*(batch+1)]
        epoch_data_points += len(x_batch)
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        epoch_loss += loss.item()
        y_out = torch.argmax(outputs.detach(), axis=1)
        correct = (y_out == y_batch).sum()
        total_correct += correct
    print("val_loss", "%.2f" % (epoch_loss/(batch + 1)), "val_acc", "%.2f" % (100 * total_correct / len(val_x)))
    return (epoch_loss/(batch + 1))

train()
test()

epoch 0
epoch_loss: 0.68, epoch_accuracy: 56.73: 100%|██████████| 293/293 [00:07<00:00, 38.58it/s]
final epoch_loss 0.68 final epoch_acc 56.56
val_loss 0.67 val_acc 55.99
0.6652491741320666 patience 3
epoch 1
epoch_loss: 0.63, epoch_accuracy: 66.22: 100%|██████████| 293/293 [00:05<00:00, 56.77it/s]
final epoch_loss 0.63 final epoch_acc 66.02
val_loss 0.55 val_acc 68.46
0.5501136884969824 patience 3
epoch 2
epoch_loss: 0.50, epoch_accuracy: 76.77: 100%|██████████| 293/293 [00:05<00:00, 56.97it/s]
final epoch_loss 0.50 final epoch_acc 76.54
val_loss 0.38 val_acc 82.38
0.3839172256343505 patience 3
epoch 3
epoch_loss: 0.43, epoch_accuracy: 80.32: 100%|██████████| 293/293 [00:05<00:00, 56.96it/s]
final epoch_loss 0.43 final epoch_acc 80.09
val_loss 0.40 val_acc 81.16
0.3839172256343505 patience 2
epoch 4
epoch_loss: 0.40, epoch_accuracy: 82.02: 100%|██████████| 293/293 [00:05<00:00, 56.91it/s]
final epoch_loss 0.40 final epoch_acc 81.78
val_loss 0.37 val_acc 83.24
0.3747382632949773 patien

In [ ]:
sample_freq = 360

def getHeartBeat(signal_data):
    m = signal_data.mean()
    std = signal_data.std()
    temp = (signal_data-m)/std
    binary = (temp > 2).astype(int)
    count = 0
    found_peak = False
    max_val = -1e10
    inds = []
    for i, val in enumerate(binary):
        if val > 0 and not found_peak:
            found_peak = True
            max_val = val
            inds.append(i)
        elif val == 0 and found_peak:
            found_peak = False
            count += 1
    d_inds = []
    # print(inds[:10])
    for i, val in enumerate(inds[:-1]):
        d_inds.append(inds[i+1] - val)
    d_inds = np.array(d_inds)
    avg_distance = d_inds.mean()
    avg_hb = sample_freq * 60/avg_distance
    # print("average hb per min", avg_hb)
    return inds

nums = ['100','101','102','103','104','105','106','107','108','109','111','112','113','114','115','116','117','118',
        '119','121','122','123','124','200','201','202','203','205','207','208','209','210','212','213','214','215',
        '217','219','220','221','222','223','228','230','231','232','233','234']




print(1/0)
for num in nums:
    # if num != "100":
    #     continue
    ann = rdann(num, extension='atr', pn_dir='mitdb')
    ann_indices = ann.sample
    ann_labels = ann.symbol
    print(ann_labels[-10:])
    count = 0
    weird = []
    for symbol in ann.symbol:
        if symbol != "N":
            count += 1
            weird.append(symbol)
    print(weird)
    print(count)
    print(ann_label_table.columns)
    record = rdrecord(num, smooth_frames= True, pn_dir='mitdb')
    test = record.p_signal
    signals0 = np.nan_to_num(record.p_signal[:,0])
    signals1 = np.nan_to_num(record.p_signal[:,1])
    print(signals0.shape)
    beatInds = getHeartBeat(signals0)
    labelInd = 0
    trueBeatInds = []
    trueBeatLabels = []
    dx = []
    for ind, beat in enumerate(beatInds):
        for i in range(labelInd, len(ann_indices)):
            if abs(beat - ann_indices[i]) < 10:
                labelInd = i
                dx.append(abs(beat - ann_indices[i]))
                trueBeatInds.append(ann_indices[i])
                trueBeatLabels.append(ann_labels[i])
                break
    print(sum(dx)/len(dx), labelInd, len(ann_indices))
    # plt.plot(signals0[:750])
    # plt.show()
    # plt.plot(signals1[2044:2402])
    # plt.show()

    
    print("")

ZeroDivisionError: ignored